###$\color{brown}{\rm Team Members}$
## Fabian Fallas
## Maofeng Tang
## Chris Gropp
## Eduardo Ponce

###$\color{brown}{\rm Imports}$

In [ ]:
import os
import re
import copy
import pprint
import itertools
import collections
import urllib.request
import urllib.parse
import numpy as np
import seaborn as sns  # 0.10.1 (0.11 fails in barplot due to API change)
import pandas
import matplotlib.pyplot as plt

###$\color{brown}{\rm Corpus~Selection}$

In [ ]:
CORPUS_URL = {
    'The Valley of Fear': "http://www.gutenberg.org/files/3289/3289.txt",
    'A Study in Scarlet': "http://www.gutenberg.org/files/244/244.txt",
    'The Sign of the Four': "http://www.gutenberg.org/files/2097/2097.txt",
    'The Hound of the Baskervilles': "http://www.gutenberg.org/files/2852/2852.txt",
    # NOTE: This file is a compilation of adventures where "The Boscombe Valley Mystery" is Adventure 4
    'The Boscombe Valley Mystery': 'https://www.gutenberg.org/files/1661/1661.txt',
}

###$\color{brown}{\rm Characters~and~Aliases}$
* Names and aliases are case-sensitive
* Underscores are used to prevent token splits when creating variant forms

In [ ]:
CHARACTERS_NAMES = {
    'The Valley of Fear': {
        'main': [
            ['Sherlock Holmes'],
            ['Watson'],
            ['John Douglas', 'Birdy Edwards', 'Mr._Douglas', 'Steve Wilson', 'Jack McMurdo'],
            ['McGinty', 'Boss', 'Bodymaster'],
            ['Alec MacDonald', 'inspector', 'Inspector', 'Mr._Mac', 'Mac'],
            ['White Mason', 'police_officer'],
            ['Mrs._Douglas'],
            ['Cecil James Barker'],
            ['Ettie Shafter'],
            ['Fred Porlock'],
            ['Professor_Moriarty', 'professor'],
            ['Sergeant Wilson'],
            ['Ted Baldwin', 'Teddy Baldwin', 'Mr._Baldwin'],
            ['Captain Marvin'],
        ],
        'detectives': [
            ['Sherlock Holmes'],
            ['Watson'],
            ['Alec MacDonald', 'inspector', 'Inspector', 'Mr._Mac', 'Mac'],
            ['White Mason', 'police_officer'],
        ],
        'perpetrators': [
            ['Professor Moriarty', 'professor'],
            ['Ted Baldwin', 'Teddy Baldwin', 'Mr._Baldwin'],
        ],
        'suspects': [
            ['Mrs. Douglas'],
            ['Cecil James Barker'],
        ],
    },
    'A Study in Scarlet': {
        'main': [
            ['Sherlock Holmes'],
            ['Watson'],
            ['John', 'John_Ferrier'],
            ['Lucy'],
            ['Enoch Drebber'],
            ['Joseph Stangerson'],
            ['Lestrade'],
            ['Gregson'],
            ['Jefferson Hope', 'cabman'],
            ['Brigham Young'],
            ['Madame_Charpentier', 'Madame', 'Mrs._Charpentier'],
            ['Arthur_Charpentier', 'Arthur'],
        ],
        'detectives': [
            ['Sherlock Holmes'],
            ['Watson'],
            ['Gregson'],
            ['Lestrade'],
        ],
        'perpetrators': [
            ['Jefferson Hope', 'cabman'],  # Part 1
            # ['Enoch Drebber'],  # Part 2
            # ['Joseph Stangerson'],  # Part 2
        ],
        'suspects': [
            ['Enoch Drebber'],  # Part 1
            ['Joseph Stangerson'],  # Part 1
            ['Madame_Charpentier', 'Madame', 'Mrs._Charpentier'],  # Part 1
            ['Arthur_Charpentier', 'Arthur'],  # Part 1
        ],
    },
    'The Sign of the Four': {
        'main': [
            ['Sherlock Holmes'],
            ['Watson'],
            ['Mary_Morstan', 'Miss_Morstan'],
            ['Arthur_Morstan', 'Captain_Morstan'],
            ['Major_Sholto', 'Major', 'major'],
            ['Thaddeus_Sholto', 'Thaddeus'],
            ['Bartholomew_Sholto', 'Bartholomew'],
            ['Jonathan Small'],
            ['Tonga'],
        ],
        'detectives': [
            ['Sherlock Holmes'],
            ['Watson'],
        ],
        'perpetrators': [
            ['Jonathan Small'],
            ['Tonga'],
        ],
        'suspects': [
            ['Major_Sholto', 'Major', 'major'],
            ['Thaddeus_Sholto', 'Thaddeus'],
        ],
    },
    'The Hound of the Baskervilles': {
        'main': [
            ['Sherlock Holmes'],
            ['Watson'],
            ['James Mortimer', 'doctor', 'Doctor'],
            ['Charles_Baskerville', 'Sir_Charles'],
            ['Henry Baskerville', 'Sir_Henry'],
            ['Jack', 'Stapleton'],
            ['Miss_Stapleton', 'Beryl'],
            ['Barrymore', 'butler'],
            ['Selden'],
            ['Laura Lyons'],
        ],
        'detectives': [
            ['Sherlock Holmes'],
            ['Watson'],
            ['Lestrade'],
        ],
        'perpetrators': [
            ['Jack', 'Stapleton'],
        ],
        'suspects': [
            ['Barrymore', 'butler'],
            ['Laura Lyons'],
        ],
    },
    'The Boscombe Valley Mystery': {
        'main': [
            ['Sherlock Holmes'],
            ['Watson'],
            ['Lestrade'],
            ['James McCarthy'],
            ['John_Turner', 'Mr._Turner', 'Turner'],
            ['Alice', 'Miss_Turner'],
        ],
        'detectives': [
            ['Sherlock Holmes'],
            ['Watson'],
            ['Lestrade'],
        ],
        'perpetrators': [
            ['John_Turner', 'Mr._Turner', 'Turner'],
        ],
        'suspects': [
            ['James McCarthy'],
        ],
    },
}

###$\color{brown}{\rm Load~Corpus}$
Read a corpus from web page or file to start processing.

In [ ]:
def get_corpus_from_url(url):
    with urllib.request.urlopen(url) as fd:
        text = fd.read()
        try:
            return text.decode('utf-8')
        except UnicodeDecodeError:
            return text.decode('iso-8859-1')


def get_corpus_from_file(file):
    with open(file) as fd:
        return fd.read()


def get_corpus(key):
    def validate_url(url):
        parsed_url = urllib.parse.urlparse(url)
        return all([parsed_url.scheme, parsed_url.netloc, parsed_url.path])

    # Check if a filename was provided
    if os.path.isfile(key):
        return get_corpus_from_file(key)
    else:
        if key in CORPUS_URL:
            file = os.path.basename(CORPUS_URL[key])
            if os.path.isfile(file):
                return get_corpus_from_file(file)

    # Check if a URL was provided
    if validate_url(key):
        return get_corpus_from_url(key)
    else:
        if key in CORPUS_URL:
            url = CORPUS_URL[key]
            if validate_url(url):
                return get_corpus_from_url(url)

    raise Exception(f"corpus '{key}' not found")

###$\color{brown}{\rm Headings~Detection~(Regex)}$
Functions to get spans of headings:
* Gutenberg tags
* Named headings - parts, chapters, adventures
* Numbered headings
* Epilogue

In [ ]:
def get_newline_index(text):
    """Find the index of the first newline in the text.
    This is used to skip/correct one newline at beginning of headings.
    """
    match = re.match(r'[ \t\r]*\n', text)
    return match.end() if match else 0


def get_gutenberg_start_heading(text, span=None):
    """Find Gutenberg's start tag (and producer, if available).

    Notes:
        * re.match() searches at the beginning of strings, but there are
          certain character combinations that are not considered strings,
          and thus need to use re.search(), even if it is at the beginning
          of line. An example are the asterisks in the Gutenberg START
          tag.
    """
    if not span:
        span = (0, len(text))

    match = re.search(
        r'(^\s*|(\s*\n){2,})'  # pre-whitespace, no indentation
        r'\*{3}\s*'  # 3 asterisks
        r'start[^\r\n]+'  # tag text
        r'\s*\*{3}'  # 3 asterisks
        r'(\s*\nproduced by.+)?'  # producer line
        r'(\s*\n){2,}',  # post-whitespace
        text[span[0]:span[1]],
    )

    if match:
        span = match.span()
        offs = get_newline_index(text[span[0]:span[1]])
        return span[0] + offs, span[1]


def get_gutenberg_end_heading(text, span=None):
    """Find Gutenberg's end tag (and transcriber's notes, if available).

    Notes:
        * Duplicate/similar Gutenberg end tags.
        * Use a newline before transcriber note to prevent matching similar
          (but indented) notes at beginning of text.
        * Use DOTALL flag to match transcriber's notes across multiple lines.
          But be wary that using DOTALL prevents the use of '.+' for other
          cases, so use '[^\r\n]' instead.
    """
    if not span:
        span = (0, len(text))

    match = re.search(
        r'('
        r'(\s*\n){2,}'  # pre-whitespace, no indentation
        r'(original transcriber.+\s*\n)?'  # transcriber notes
        r'end[^\r\n]+'  # duplicate/similar tag text
        r')?'
        r'\s*\n'  # pre-whitespace, no indentation
        r'\*{3}\s*'  # 3 asterisks
        r"end[^\r\n]+"  # tag text
        r'\s*\*{3}'  # 3 asterisks
        r'(\s*\n){2,}',  # post-whitespace
        text[span[0]:span[1]],
        flags=re.DOTALL,
    )

    if match:
        span = match.span()
        offs = get_newline_index(text[span[0]:span[1]])
        return span[0] + offs, span[1]


def get_named_headings(text, name, span=None):
    """Find named headings with title."""
    if not span:
        span = (0, len(text))

    spans = [
        (match.start() + span[0], match.end() + span[0])
        for match in re.finditer(
            r'(^(\s*)|(\s*\n){2,})'  # pre-whitespace, no indentation
            r'('
            fr'{name}[ \t]+(\d+|[ivxlcd]+)'  # label with Arabic/Roman number
            r'(-+|\.)?'  # label-title delimiter
            r'((\s*\n){2})?'  # whitespace for titles two line apart
            r'.*(\r?\n.*)?'  # title (muti-line support)
            r'|'  # cases: name # \s* label, # name/label
            r'(\d+|[ivxlcd]+)'  # label with Arabic or Roman numbering
            r'(-+|\.)?'  # label-title delimiter
            fr'[ \t]+.*{name}.*'  # label with name
            r')'
            r'(\s*\n){2,}',  # post-whitespace
            text[span[0]:span[1]],
        )
    ]

    _spans = []
    for _span in spans:
        offs = get_newline_index(text[_span[0]:_span[1]])
        _spans.append((_span[0] + offs, _span[1]))
    return _spans


def get_numbered_headings(text, span=None):
    """Find numbered headings with no title."""
    if not span:
        span = (0, len(text))

    spans = [
        (match.start() + span[0], match.end() + span[0])
        for match in re.finditer(
            r'(^\s*|(\s*\n){2,})'  # pre-whitespace, no indentation
            fr'(\d+|[ivxlcd]+)'  # label with Arabic or Roman numbering
            r'(-+|\.)'  # label-title delimiter
            r'(\s*\n){2,}',  # post-whitespace
            text[span[0]:span[1]]
        )
    ]

    _spans = []
    for _span in spans:
        offs = get_newline_index(text[_span[0]:_span[1]])
        _spans.append((_span[0] + offs, _span[1]))
    return _spans


def get_epilogue_heading(text, span=None):
    if not span:
        span = (0, len(text))

    match = re.search(
        r'(^\s*|(\s*\n){2,})'  # pre-whitespace, no indentation
        r'epilogue'  # tag text
        r'(\s*\n){2,}',  # post-whitespace
        text[span[0]:span[1]]
    )

    if match:
        span = match.span()
        offs = get_newline_index(text[span[0]:span[1]])
        return span[0] + offs, span[1]

###$\color{brown}{\rm Regions~of~Interest~(ROI)}$
Functions to get spans of text between headings.

In [ ]:
def get_headings_map(
    text,
    headings=['part', 'chapter', 'adventure', 'epilogue', 'numbered'],
):
    """Create a list of all heading spans, guarantees at least one set
    of bounding spans.

    Args:
        headings (str, List[str]): Heading names to search for.
    """
    if not isinstance(headings, (list, tuple, set)):
        _headings = [headings]
    else:
        _headings = copy.deepcopy(headings)

    headings_map = {}
    _headings_map = {}

    # Always available heading, all text
    text_heading = '_text_'

    # Ensure there is always a begin "span"
    start_span = get_gutenberg_start_heading(text)
    if not start_span:
        start_span = 0, 0

    # Ensure there is always an end "span"
    end_span = get_gutenberg_end_heading(text)
    if not end_span:
        end_span = len(text), len(text)
    headings_map[text_heading] = [start_span, end_span]
    if text_heading in _headings:
        _headings.remove(text_heading)

    # Optional
    span = get_epilogue_heading(text)
    if span:
        heading = 'epilogue'
        _headings_map[heading] = [span, headings_map[text_heading][1]]
        if heading in _headings:
            headings_map[heading] = _headings_map[heading]
            _headings.remove(heading)

    # Optional
    spans = get_numbered_headings(text)
    if spans:
        heading = 'numbered'
        _headings_map[heading] = [*spans, headings_map[text_heading][1]]
        if heading in _headings:
            headings_map[heading] = _headings_map[heading]
            _headings.remove(heading)

    # Optional
    for heading in _headings:
        spans = get_named_headings(text, heading)
        if spans:
            headings_map[heading] = spans
            if 'epilogue' in _headings_map:
                headings_map[heading].append(_headings_map['epilogue'][0])
            else:
                headings_map[heading].append(headings_map[text_heading][1])
    return headings_map


def select_rois_spans(spans, n=None):
    if n is None:
        _spans = [
            (spans[i][1], spans[i + 1][0])
            for i in range(len(spans) - 1)
        ]
    else:
        _spans = [
            (spans[i - 1][1], spans[i][0])
            for i in ([n] if isinstance(n, int) else n)
            if i >= 1 and i < (len(spans))
        ]
    return _spans


def remove_embedded_spans(spans):
    non_embedded_spans = copy.deepcopy(spans)
    for i in range(len(spans)):
        span = spans[i]
        for j in range(i + 1, len(spans)):
            _span = spans[j]
            if span[0] >= _span[0] and span[1] <= _span[1]:
                non_embedded_spans.remove(span)
                break
            elif span[1] > _span[1]:
                break
    non_embedded_spans.sort()
    return non_embedded_spans


def get_nonoverlapped_spans(spans, *, join=True):
    """Remove fully embedded spans and join overlapped spans."""
    non_embedded_spans = remove_embedded_spans(spans)
    non_embedded_spans = remove_embedded_spans(non_embedded_spans[::-1])
    if not join:
        return non_embedded_spans

    joined_spans = []
    for span in non_embedded_spans:
        for _span in non_embedded_spans:
            if span != _span:
                joined_span = None
                if span[0] >= _span[0] and span[0] <= _span[1]:
                    joined_span = (_span[0], span[1])
                elif span[1] >= _span[0] and span[1] <= _span[1]:
                    joined_span = (span[0], _span[1])
                if joined_span:
                    if joined_span not in joined_spans:
                        joined_spans.append(joined_span)
                    break
        else:
            joined_spans.append(span)

    nonoverlap_spans = sorted(joined_spans)

    # Recurse until condition is satisfied
    if nonoverlap_spans == spans:
        return nonoverlap_spans
    return get_nonoverlapped_spans(nonoverlap_spans)


def contains_span(spans, span):
    """Validate if a span is contained in a collection of spans."""
    for _span in spans:
        if span[0] >= _span[0] and span[1] <= _span[1]:
            return True
    return False


def get_rois(text, name=None, *, n=None, headings_map=None):
    """Get span bounding a ROI.

    Args:
        name (str): ROI

        n (int, Iterable[int]): Number of ROI, [1,N]
    """
    if not headings_map:
        headings_map = get_headings_map(text)

    # Always available heading, all text
    text_heading = '_text_'

    rois = []
    if not name:
        rois = [(
            headings_map[text_heading][0][1],
            headings_map[text_heading][1][0],
        )]
    elif name in headings_map:
        rois = select_rois_spans(headings_map[name], n)

    # If necessary, skip last inner heading
    _rois = []
    for roi in rois:
        value = roi[1]
        for spans in headings_map.values():
            for span in spans:
                if roi[1] > span[0] and roi[1] <= span[1]:
                    value = span[0]
        _rois.append((roi[0], value))
    return _rois



def get_roi(text, name, span=None, *, n=None):
    if not span:
        spans = get_rois(text, name, n=n)
    else:
        spans = [
            (_span[0] + span[0], _span[1] + span[0])
            for _span in get_rois(text[span[0]:span[1]], name, n=n)
        ]
    return spans


def get_text_from_span(text, span=None):
    if not span:
        span = (0, len(text))
    elif isinstance(span[0], int):
        span = [span]

    roi = ''
    for _span in span:
        roi += text[_span[0]:_span[1]]
    return roi


def get_text(text, span=None, *, n=None):
    return get_rois(text)


def get_parts(text, span=None, *, n=None):
    return get_roi(text, 'part', span, n=n)


def get_chapters(text, span=None, *, n=None):
    return get_roi(text, 'chapter', span, n=n)


def get_adventures(text, span=None, *, n=None):
    return get_roi(text, 'adventure', span, n=n)


def get_numbered_sections(text, span=None, *, n=None):
    return get_roi(text, 'numbered', span, n=n)


def get_epilogue(text, span=None):
    return get_roi(text, 'epilogue', span)

###$\color{brown}{\rm Tokenization~(Regex)}$
Regexes to get text decomposition:
* Paragraphs
* Sentences
* Tokens

In [ ]:
def tokenize(text, span=None, regex=r'\w', *, use_remaining=False):
    def _get_tokens(text):
        return [
            match.span()
            for match in re.finditer(regex, text)
        ]

    if not span:
        span = (0, len(text))

    # Get tokens from text
    # Add base offset to tokens' spans
    tokens = [
        (tok_span[0] + span[0], tok_span[1] + span[0])
        for tok_span in _get_tokens(text[span[0]:span[1]])
    ]

    if use_remaining:
        if tokens:
            # Extend last token to end of text
            tokens[-1] = tokens[-1][0], span[1]
        else:
            # Consider all text as the token
            tokens = [span]

    return tokens


def select_spans(spans, n=None):
    if n is None:
        _spans = spans
    else:
        _spans = [
            spans[i - 1]
            for i in ([n] if isinstance(n, int) else n)
            if i >= 1 and i <= (len(spans))
        ]
    return _spans


def get_paragraphs(text, span=None, *, n=None):
    spans = tokenize(
        text,
        span,
        r'('
        r'([^\r\n]+\r?\n)+'  # (regular text with newline)+
        r'('
        r'(\r?\n)+'  # (newline)+
        r'[^a-zA-Z]'  # non-alpha character: quote, number, etc.
        r')?'  # handles case of multiple newlines but still same paragraph
        r')+',  # (full regex)+
        use_remaining=True,
    )
    return select_spans(spans, n)


def get_sentences(text, span=None, *, n=None):
    spans = tokenize(
        text,
        span,
        r'('
        r'([^\.\r\n;M!]+(\r?\n)?)+'
        r'(.")?'
        r'(M[rR][sS]?\.\s)?'
        r'M?'
        r')+'
        r'|'
        r'M[rR][sS]?'
        r'\.\s'
        r'([^\.;M!]+(.")?(M[rR][sS]?\.\s)?(M)?)+',
    )
    return select_spans(spans, n)


def get_tokens(text, span=None, *, n=None):
    spans = tokenize(
        text,
        span,
        r'\w+'  # compound alphanumeric words
        r'('
        r"'\w+"  # contractions
        r'|(-\w+)+'  # tokens with inlined dashes
        r')'
        r'|\w+'  # single alphanumeric words
        r'|\$?-?\d+(,\d+)*(\.\d+)?',  # numbers, decimals, monetary
    )
    return select_spans(spans, n)


def get_conversations(text, span=None, *, n=None):
    spans = tokenize(text, span, r'"[^"]+"')
    return select_spans(spans, n)

###$\color{brown}{\rm Named~Entity~Recognition~(NER)}$
Find names of characters/places.

In [ ]:
def abbreviate_entity(entity):
    """Trim names to fit in plots (F. LastName)."""
    entity_parts = entity.split()
    if len(entity_parts) == 1:
        abbrv = entity
    else:
        # F. LastName
        abbrv = f'{entity_parts[0][0]}. {entity_parts[-1]}'
        # FirstName L.
        # abbrv = f'{entity_parts[0]} {entity_parts[-1][0]}.'
    return abbrv


def get_entity_variants(entity, *, invalids=[]):
    """Given a string with whitespace, return all combinations."""
    if '_' in entity:
        entity_parts = [entity.replace('_', ' ')]
    else:
        entity_parts = entity.split()

    variants = []
    for i in range(len(entity_parts)):
        for j in range(i+1, len(entity_parts)+1):
            variant = ' '.join(entity_parts[i:j])
            if variant not in invalids:
                variants.append(variant)
    return variants


def search_entity(entities, text, span=None, *, invalids=[]):
    if not isinstance(entities, (list, tuple, set)):
        entities = [entities]

    if span is None:
        span = (0, len(text))

    entities_map = collections.defaultdict(list)
    for entity in entities:
        entity_variants = get_entity_variants(
            entity,
            invalids=invalids,
        )
        entity_map = get_keywords_map(text, span, entity_variants)
        if entity_map:
            for spans in entity_map.values():
                entities_map[entity].extend(spans)

    return {entities[0]: get_nonoverlapped_spans(list(itertools.chain.from_iterable(entities_map.values())))}

def get_max_frequency_from_nested_map(nested_freq_map):
    ymax = 0
    for _, freq_map in nested_freq_map.items():
        ymax = max(ymax, max(freq_map.values()))
    return ymax


def get_max_frequency_from_nested2_map(nested2_freq_map):
    ymax = 0
    for _, xmap in nested2_freq_map.items():
        ymax = max(ymax, get_max_frequency_from_nested_map(xmap))
    return ymax


def ner_story_with_chapters(story, span=None):
    story_spans = {}
    story_counts = {}
    corpus = get_corpus(story)
    text = corpus.lower()
    for character_type, character_list in CHARACTERS_NAMES[story].items():
        story_spans[character_type] = {}
        story_counts[character_type] = {}
        for n, chp_span in enumerate(get_chapters(text, span), start=1):
            story_spans[character_type][n] = {}
            story_counts[character_type][n] = {}
            for character_names in character_list:
                story_spans[character_type][n].update(search_entity(character_names, corpus, chp_span))
            story_counts[character_type][n] = convert_spans_to_counts_map(story_spans[character_type][n])
    return story_spans, story_counts


def ner_story(story, span=None):
    story_spans = {}
    story_counts = {}
    corpus = get_corpus(story)
    for character_type, character_list in CHARACTERS_NAMES[story].items():
        story_spans[character_type] = {}
        story_counts[character_type] = {}
        story_spans[character_type][0] = {}
        story_counts[character_type][0] = {}
        for character_names in character_list:
            story_spans[character_type][0].update(search_entity(character_names, corpus, span))
            story_counts[character_type][0] = convert_spans_to_counts_map(story_spans[character_type][0])
    return story_spans, story_counts


def ner_story_with_paragraphs(story, span=None):
    story_spans = {}
    story_counts = {}
    corpus = get_corpus(story)
    text = corpus.lower()
    for character_type, character_list in CHARACTERS_NAMES[story].items():
        story_spans[character_type] = {}
        story_counts[character_type] = {}
        for n, par_span in enumerate(get_paragraphs(text, span), start=1):
            story_spans[character_type][n] = {}
            story_counts[character_type][n] = {}
            for character_names in character_list:
                story_spans[character_type][n].update(search_entity(character_names, corpus, par_span))
            story_counts[character_type][n] = convert_spans_to_counts_map(story_spans[character_type][n])
    return story_spans, story_counts


def plot_ner_counts_story_with_chapters(story, counts, *, show=False):
    ylim = (0, get_max_frequency_from_nested2_map(counts))
    for character_type in CHARACTERS_NAMES[story].keys():
        ns = int(np.ceil(np.sqrt(len(counts[character_type]))))
        fig, axes = plt.subplots(ns, ns, constrained_layout=True)
        axes = trim_axes(axes, len(counts[character_type]))
        for ax, (chp, freq) in zip(axes, counts[character_type].items()):
            # Abbreviate names for plots
            labels = []
            data = []
            for k, v in freq.items():
                labels.append(abbreviate_entity(k))
                data.append(v)
            barplot(data, labels, xlabel=f'Chapter {chp}', ylim=ylim, ax=ax)
        print(f'{story} - {character_type}')
        if show:
            plt.show()


def plot_ner_counts_story(story, counts, *, show=False):
    ylim = (0, get_max_frequency_from_nested2_map(counts))
    for character_type in CHARACTERS_NAMES[story].keys():
        freq = counts[character_type][0]

        # Abbreviate names for plots
        labels = []
        data = []
        for k, v in freq.items():
            labels.append(abbreviate_entity(k))
            data.append(v)

        barplot(data, labels, xlabel='', ylim=ylim)
        print(f'{story} - {character_type}')
        if show:
            plt.show()


def plot_ner_counts_story_with_paragraphs(story, counts, *, show=False):
    ylim = (0, get_max_frequency_from_nested2_map(counts))
    for character_type in CHARACTERS_NAMES[story].keys():
        ns = int(np.ceil(np.sqrt(len(counts[character_type]))))
        fig, axes = plt.subplots(ns, ns, constrained_layout=True)
        axes = trim_axes(axes, len(counts[character_type]))
        for ax, (par, freq) in zip(axes, counts[character_type].items()):
            # Abbreviate names for plots
            labels = []
            data = []
            for k, v in freq.items():
                labels.append(abbreviate_entity(k))
                data.append(v)
            barplot(data, labels, xlabel=f'Paragraph {par}', ylim=ylim, ax=ax)
        print(f'{story} - {character_type}')
        if show:
            plt.show()

###$\color{brown}{\rm Frequency~Count}$
For vocabulary and keywords/entities

In [ ]:
def get_keywords_map(text, span=None, keywords=[], *, spans_map=None):
    if spans_map is None:
        spans_map = collections.defaultdict(list)

    # NOTE: Allow cases where token is a prefix/compound of a longer token
    for kw in keywords:
        spans_map[kw].extend(tokenize(
            text,
            span,
            fr'(?<![a-zA-Z0-9]){kw}',
        ))
    return spans_map


def generate_frequency_map(spans_map, *, threshold=None):
    freq = collections.defaultdict(int)
    for k, v in spans_map.items():
        if threshold is None or len(v) >= threshold:
            freq[k] = len(v)
    return freq


def convert_spans_to_counts_map(spans_map):
    return {
        key: len(spans)
        for key, spans in spans_map.items()
    }


def get_vocabulary(text, span=None, *, vocab=None):
    return (
        list(vocab) if vocab else []
    ) + [
        get_text_from_span(text, token_span)
        for token_span in get_tokens(text, span)
    ]


def get_vocabulary_map(text, span=None, *, spans_map=None):
    if spans_map is None:
        spans_map = collections.defaultdict(list)

    for token_span in get_tokens(text, span):
        token = get_text_from_span(text, token_span)
        spans_map[token].append(token_span)
    return spans_map


def get_frequent_items(data, n=10):
    d = collections.Counter(data)
    return dict(d.most_common(n))

###$\color{brown}{\rm First~Time~Occurrences}$

In [ ]:
def get_first_occurrences(story_spans):
    """Get spans of first occurrences."""
    firstoccurrences = {}
    for character_type, sections in story_spans.items():
        firstoccurrences[character_type] = {}
        for section, characters in sections.items():
            for character in characters.keys():
                spans = story_spans[character_type][section][character]
                if character not in firstoccurrences[character_type] and spans:
                    firstoccurrences[character_type][character] = spans[0]
    return firstoccurrences


def get_span_location_with_chapters(text, span, search_span, *, verbose=False):
    """Get text elements of span occurrence."""
    location = {'chapter': None, 'paragraph': None, 'sentence': None, 'sentence_chp': None}
    for ichp, chp_span in enumerate(get_chapters(text, span), start=1):
        csent = 0  # chapter based sentence
        for ipar, par_span in enumerate(get_paragraphs(text, chp_span), start=1):
            for isent, sent_span in enumerate(get_sentences(text, par_span), start=1):
                csent += 1
                if verbose:
                    print(f'Chapter {ichp}, Paragraph {ipar}, Sentence {isent}, Sentence (Chp) {csent}')
                    print(get_text_from_span(text, sent_span))
                if search_span[0] >= sent_span[0] and search_span[1] <= sent_span[1]:
                    location['chapter'] = ichp
                    location['paragraph'] = ipar
                    location['sentence'] = isent
                    location['sentence_chp'] = csent
                    return location


def get_span_location(text, span, search_span, *, verbose=False):
    """Get text elements of span occurrence."""
    location = {'paragraph': None, 'sentence': None, 'sentence_text': None}
    csent = 0  # text based sentence
    for ipar, par_span in enumerate(get_paragraphs(text, span), start=1):
        for isent, sent_span in enumerate(get_sentences(text, par_span), start=1):
            csent += 1
            if verbose:
                print(f'Paragraph {ipar}, Sentence {isent}, Sentence (Text) {csent}')
                print(get_text_from_span(text, sent_span))
            if search_span[0] >= sent_span[0] and search_span[1] <= sent_span[1]:
                location['paragraph'] = ipar
                location['sentence'] = isent
                location['sentence_text'] = csent
                return location

###$\color{brown}{\rm Neighboring~Words}$

In [ ]:
STOPWORDS = {
    'yours', 'do', 'might', 'although', 'all', 'he', "'s", 'wherein',
    'themselves', 'used', 'anyone', 'others', 'whom', 'off', 'doing',
    'she', 'no', 'even', 'behind', 'them', 'done', 'none', 'beside',
    'whither', 'yet', 'but', 'perhaps', 'one', 'its', 'thereby', 'on',
    'hereby', 'nine', 'just', 'ca', 'front', "'d", 'also', 'across',
    'seems', 'towards', 'together', 'itself', 'four', 'that', 'why', 'it',
    'so', 'three', 'down', 'keep', 'sixty', 'empty', 'above', 'whose',
    'between', 'hence', 'two', 'be', 'however', 'nothing', 'are', 'whence',
    're', 'beyond', 'being', 'nowhere', 'always', 'was', 'via', 'mine',
    'ourselves', 'quite', 'when', 'only', 'should', 'amongst', 'before',
    'from', 'any', 'most', 'how', 'same', 'if', 'latter', 'something',
    'fifty', 'an', 'have', 'who', 'too', 'up', 'ours', 'which',
    'beforehand', 'else', 'sometimes', 'to', 'either', 'really', 'take',
    'such', 'go', 'again', 'into', 'a', 'per', 'anyhow', 'anything',
    'elsewhere', 'hundred', 'afterwards', 'we', 'since', 'yourselves',
    'both', 'top', 'formerly', "'m", 'her', 'alone', 'whereas', 'becoming',
    'what', "n't", 'back', 'or', "'ll", 'never', 'everyone', 'various',
    'then', 'over', 'against', 'twelve', 'herself', 'those', 'they',
    'became', 'thereafter', 'forty', 'latterly', 'seeming', 'see', 'of',
    'much', 'thereupon', 'regarding', 'get', 'give', 'by', 'indeed', "'ve",
    'thus', 'part', 'can', 'still', 'is', 'nor', 'first', 'eight',
    'nevertheless', 'another', 'along', 'i', 'former', 'someone',
    'without', 'noone', 'whoever', 'becomes', 'about', 'through', 'unless',
    'fifteen', 'namely', 'anywhere', 'will', 'as', 'each', 'during', 'few',
    'become', 'their', 'hereafter', 'could', 'third', 'thru', 'somehow',
    'in', 'bottom', 'am', 'seem', 'otherwise', 'here', 'several', 'say',
    'would', 'our', 'for', 'due', 'move', 'somewhere', 'under', 'himself',
    'already', 'with', 'except', 'mostly', 'amount', 'more', 'you', 'his',
    'almost', 'every', 'upon', 'throughout', 'often', 'below', 'been',
    'whatever', 'eleven', 'whole', 'within', 'cannot', 'five', 'him',
    'hers', 'yourself', 'next', 'once', 'around', "'re", 'thence', 'using',
    'does', 'until', 'were', 'make', 'onto', 'us', 'your', 'while',
    'because', 'some', 'show', 'full', 'everything', 'did', 'after',
    'call', 'now', 'the', 'meanwhile', 'many', 'whereupon', 'everywhere',
    'me', 'name', 'not', 'seemed', 'least', 'must', 'six', 'less',
    'serious', 'there', 'whereby', 'whether', 'own', 'and', 'whereafter',
    'has', 'may', 'neither', 'where', 'ever', 'wherever', 'made',
    'moreover', 'well', 'myself', 'among', 'please', 'other', 'out',
    'this', 'therein', 'rather', 'though', 'hereupon', 'besides', 'had',
    'at', 'twenty', 'ten', 'these', 'my', 'than', 'side', 'nobody', 'very',
    'last', 'sometime', 'toward', 'herein', 'whenever', 'further',
    'anyway', 'enough', 'therefore', 'put',
}


# Get neighbor words (even across sentences)
def get_neighbor_words_for_character(text, spans, characters, alias_list, *, n=3, stopwords=None):
    neighbors = {}
    for character, cspans in characters.items():
        neighbors[character] = {}
        neighbors[character]['before'] = collections.defaultdict(int)
        neighbors[character]['after'] = collections.defaultdict(int)
        # neighbors[character]['before_spans'] = collections.defaultdict(list)
        # neighbors[character]['after_spans'] = collections.defaultdict(list)

        aliases = []
        for clist in alias_list:
            if character == clist[0]:
                aliases = [alias.lower() for alias in clist]
                break

        # Combine token and character spans 
        tmp = copy.deepcopy(spans)
        tmp.extend(cspans)
        spans2 = get_nonoverlapped_spans(tmp, join=False)

        last_idx = 0
        for cspan in cspans:
            try:
                idx = spans2.index(cspan, last_idx)
                last_idx = idx
            except ValueError:
                continue

            # Hack to dismiss names as neighbors
            prev_n = 0
            _n = 1
            while prev_n < n:
                prev_idx = idx - _n
                _n += 1
                if prev_idx >= 0:
                    token = spans2[prev_idx]
                    word = get_text_from_span(text, token)

                    valid = True
                    for alias in aliases:
                        if word in alias or word.replace(' ', '_') in alias:
                            valid = False
                            break

                    if valid and stopwords:
                        valid = word not in stopwords

                    if not valid:
                        continue
                    
                    prev_n += 1
                    neighbors[character]['before'][word] += 1
                    # neighbors[character]['before_spans'][word].append(token)
                else:
                    break

            after_n = 0
            _n = 1
            while after_n < n:
                after_idx = idx + _n
                _n += 1
                if after_idx < len(spans2):
                    token = spans2[after_idx]
                    word = get_text_from_span(text, token)

                    valid = True
                    for alias in aliases:
                        if word in alias or word.replace(' ', '_') in alias:
                            valid = False
                            break

                    if valid and stopwords:
                        valid = word not in stopwords

                    if not valid:
                        continue
                    
                    after_n += 1
                    neighbors[character]['after'][word] += 1
                    # neighbors[character]['after_spans'][word].append(token)
                else:
                    break

    return neighbors

###$\color{brown}{\rm Visualization}$

In [ ]:
def trim_axes(axes, n):
    axes = axes.flat
    for ax in axes[n:]:
        ax.remove()
    return axes[:n]


def visualize_co_occurrence(data, keywords_rows, closeness_words_columns):
    # Create a dataframe from the provided data
    df = pandas.DataFrame(
        data,
        index=keywords_rows,
        columns=closeness_words_columns,
    )

    # Set plot size according to the number of cols and rows
    plt.figure(figsize=(len(keywords_rows), len(closeness_words_columns)))

    # Set color of heatmap
    # hdl = sns.heatmap(df, cmap="YlGnBu", annot=True, annot_kws={'fontsize': 20}, linewidths=.5, linecolor='black', square=True)
    hdl = sns.heatmap(df, cmap="YlGnBu", annot=True, annot_kws={'fontsize': 20}, linewidths=.5, linecolor='black', cbar=False, square=True)

    # Rotate text
    loc_x, labels_x = plt.xticks()
    loc_y, labels_y = plt.yticks()
    hdl.set_xticklabels(labels_x, rotation=0, fontsize=20)
    hdl.set_yticklabels(labels_y, rotation=0, fontsize=20)
    # sns.set(font_scale=1)


def barplot(data, labels, *, xlabel='Word', ylabel='Frequency', ylim=None, ax=None):
    # Prepare the data to pandas
    data_ = [labels, data]
    data_ = np.asarray(data_).transpose()

    # Create a dataframe from the provided data
    df = pandas.DataFrame(data_, columns=[xlabel, ylabel])
    hdl = sns.barplot(x=xlabel, y=ylabel, data=df, palette="Blues_d", ax=ax)
    plt.setp(hdl.get_xticklabels(), rotation=0)
    # plt.setp(hdl.get_xticklabels(), rotation=15)
    plt.rc('xtick', labelsize=16)
    plt.rc('ytick', labelsize=16)
    plt.rc('axes', labelsize=20)

    if ylim:
        if ax:
            ax.set(ylim=ylim)
        else:
            plt.ylim(*ylim)

# Co-occurrence matrix of perpetrators
Co-occurrence of perpetrators across all novels against Sherlock Holmes, Watson, and neighboring words.

In [ ]:
# Example of expected format of data that the method needs
rows_example = ['Holmes', 'Watson', 'criminal', 'treasure', 'man', 'poison', 'killed']

columns_example = ["P. Moriarty", "T. Baldwin", "J. Hope","J. Small","Tonga","J. Stapleton","J. Turner"]
data_example = [[1,1,0,0,0,0,0],[0,0,0,0,1,0,0],[1,1,0,0,0,1,0],[0,0,0,1,1,0,0],[0,0,0,1,0,0,1],[0,0,0,0,1,0,0],[0,0,0,0,2,0,0]]

# Plot
visualize_co_occurrence(data_example,rows_example,columns_example)
plt.show()

Example of word count histogram

In [ ]:
# Example of expected format of data that the method needs
labels_example = ["murder", "guilty", "suspect", "assassin"]
data_example = [5,8,3,1]

# Plot
barplot(data_example,labels_example)
plt.show()

###$\color{brown}{\rm Drivers}$

# Get text from stories

In [ ]:
corpus = get_corpus('The Valley of Fear')
corpus_l = corpus.lower()
spans = get_text(corpus_l)
text = get_text_from_span(corpus_l, spans[0])
print(text[:100])
print('-' * 80)

corpus = get_corpus('A Study in Scarlet')
corpus_l = corpus.lower()
spans = get_parts(corpus_l, n=1)  # n = Part
text = get_text_from_span(corpus_l, spans[0])
print(text[:100])
print('-' * 80)

corpus = get_corpus('The Sign of the Four')
corpus_l = corpus.lower()
spans = get_text(corpus_l)
text = get_text_from_span(corpus_l, spans[0])
print(text[:100])
print('-' * 80)

corpus = get_corpus('The Hound of the Baskervilles')
corpus_l = corpus.lower()
spans = get_text(corpus_l)
text = get_text_from_span(corpus_l, spans[0])
print(text[:100])
print('-' * 80)

corpus = get_corpus('The Boscombe Valley Mystery')
corpus_l = corpus.lower()
spans = get_adventures(corpus_l, n=4)
text = get_text_from_span(corpus_l, spans[0])
print(text[:100])

# Negative words frequency

In [ ]:
def get_negative_words(negative_words):
    """
    Get list of negative words
    """
    return [negative_words[match.start():match.end()] for match in re.finditer(r'(?<=\n)[a-zA-Z]+(?=\n)', negative_words)]
    # return [negative_words[match.start():match.end()] for match in re.finditer(r'(?<=\n)[a-zA-Z]+(?=\n)', negative_words)]


def get_top_negative_words(corpus, negative_list, *, top=20):
    """
    Get top list of negative words
    """
    frequency_list = []
    for word in negative_list:
        result_list = [match.span() for match in re.finditer(fr'(?<![a-zA-Z0-9]){word}', corpus)]
        frequency_list.append(len(result_list))
    
    # Sort results
    temp_list = np.asarray(list(zip(negative_list,frequency_list)), dtype = [('word', np.unicode_, 16), ('frequency', int)] )
    sorted_list = np.sort(temp_list, order='frequency', kind="quicksort")
    sorted_list = sorted_list[::-1]

    return sorted_list[:top]

In [ ]:
# Get negative words
url = "https://raw.githubusercontent.com/edponce/DoyleInvestigators/develop/negative-words.txt"
negative_words = get_corpus_from_url(url)
result_negative = get_negative_words(negative_words)

In [ ]:
story = 'The Valley of Fear'

# Get corpus (use all text between Gutenberg tags)
corpus = get_corpus(story)
corpus_l = corpus.lower()
text = get_text_from_span(corpus_l, get_text(corpus_l)[0])

# Get count
results = get_top_negative_words(text, result_negative, top=20)
print(results)

In [ ]:
story = 'A Study in Scarlet'

# Get corpus (use all text between Gutenberg tags)
corpus = get_corpus(story)
corpus_l = corpus.lower()
spans = get_parts(corpus_l, n=1) # n = Part
text = get_text_from_span(corpus_l, spans[0])

# Get count
results = get_top_negative_words(text, result_negative, top=20)
print(results)

In [ ]:
story = 'The Sign of the Four'

# Get corpus (use all text between Gutenberg tags)
corpus = get_corpus(story)
corpus_l = corpus.lower()
text = get_text_from_span(corpus_l, get_text(corpus_l)[0])

# Get count
results = get_top_negative_words(text, result_negative, top=20)
print(results)

In [ ]:
story = 'The Hound of the Baskervilles'

# Get corpus (use all text between Gutenberg tags)
corpus = get_corpus(story)
corpus_l = corpus.lower()
text = get_text_from_span(corpus_l, get_text(corpus_l)[0])

# Get count
results = get_top_negative_words(text, result_negative, top=20)
print(results)

In [ ]:
story = 'The Boscombe Valley Mystery'

# Get corpus (use all text between Gutenberg tags)
corpus = get_corpus(story)
corpus_l = corpus.lower()
spans = get_adventures(corpus_l, n=4)
text = get_text_from_span(corpus_l, spans[0])

# Get count
results = get_top_negative_words(text, result_negative, top=20)
print(results)

# Crime Detection

In [ ]:
def crime_story_with_chapters(keywords, text, span=None):
    story_spans = {}
    story_counts = {}
    for n, chp_span in enumerate(get_chapters(text, span), start=1):
        story_spans[n] = {}
        story_counts[n] = {}
        for kw in keywords:
            story_spans[n].update(search_entity(kw, text, chp_span))
        story_counts[n] = convert_spans_to_counts_map(story_spans[n])
    return story_spans, story_counts


def crime_story(keywords, text, span=None):
    story_spans = {}
    story_counts = {}
    story_spans[0] = {}
    story_counts[0] = {}
    for kw in keywords:
        story_spans[0].update(search_entity(kw, text, span))
    story_counts[0] = convert_spans_to_counts_map(story_spans[0])
    return story_spans, story_counts


def get_first_crime(story_spans):
    """Get spans of first crime occurrences."""
    firstoccurrences = {}
    for section, keywords in story_spans.items():
        for kw, spans in keywords.items():
            if kw not in firstoccurrences and spans:
                firstoccurrences[kw] = spans[0]
    return firstoccurrences


def plot_crime_counts_story_with_chapters(story, counts, *, show=False):
    ylim = (0, get_max_frequency_from_nested_map(counts))
    ns = int(np.ceil(np.sqrt(len(counts))))
    fig, axes = plt.subplots(ns, ns, constrained_layout=True)
    axes = trim_axes(axes, len(counts))
    for ax, (chp, freq) in zip(axes, counts.items()):
        # Abbreviate names for plots
        labels = []
        data = []
        for k, v in freq.items():
            labels.append(k)
            data.append(v)
        barplot(data, labels, xlabel=f'Chapter {chp}', ylim=ylim, ax=ax)
    print(f'{story}')
    if show:
        plt.show()


def plot_ner_counts_story(story, counts, *, show=False):
    ylim = (0, get_max_frequency_from_nested2_map(counts))
    for character_type in CHARACTERS_NAMES[story].keys():
        freq = counts[character_type][0]

        # Abbreviate names for plots
        labels = []
        data = []
        for k, v in freq.items():
            labels.append(abbreviate_entity(k))
            data.append(v)

        barplot(data, labels, xlabel='', ylim=ylim)
        print(f'{story} - {character_type}')
        if show:
            plt.show()

In [ ]:
# Search keywords
CRIME_KEYWORDS = [
    'dead', 'death', 'murder', 'crime', 'hurt', 'blood', 'treasure', 'suffer',
    'guilty', 'assassin', 'pain', 'theft', 'steal', 'victim', 'poison',
    'gunshot', 'criminal', 'wound', 'attack',
]

### The Valley of Fear, The Sign of the Four, The Hound of the Baskervilles

In [ ]:
story = 'The Valley of Fear'
#story = 'The Sign of the Four'
#story = 'The Hound of the Baskervilles'

# Get corpus span
corpus = get_corpus(story)
corpus_l = corpus.lower()
spans = get_text(corpus_l)
span = spans[0]


# Full story
story_spans, story_counts = crime_story_with_chapters(CRIME_KEYWORDS, corpus_l, span)
plot_crime_counts_story_with_chapters(story, story_counts, show=True)

pp = pprint.PrettyPrinter(indent=2)
#pp.pprint(story_spans)
pp.pprint(story_counts)


# Get span of first occurrence
first_occurrences = get_first_crime(story_spans)
# print(first_occurrences)

# Find location of first span
first_locations = {}
for kw, kw_span in first_occurrences.items():
    location = get_span_location(corpus_l, span, kw_span, verbose=False)
    first_locations[kw] = location

print(story)
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(first_locations)

### A Study in Scarlet


In [ ]:
story = 'A Study in Scarlet'

# Get corpus span
corpus = get_corpus(story)
corpus_l = corpus.lower()
spans = get_parts(corpus_l, n=1)  # n = Part
span = spans[0]


# Full story
story_spans, story_counts = crime_story_with_chapters(CRIME_KEYWORDS, corpus_l, span)
plot_crime_counts_story_with_chapters(story, story_counts, show=True)

pp = pprint.PrettyPrinter(indent=2)
#pp.pprint(story_spans)
pp.pprint(story_counts)


# Get span of first occurrence
first_occurrences = get_first_crime(story_spans)
# print(first_occurrences)

# Find location of first span
first_locations = {}
for kw, kw_span in first_occurrences.items():
    location = get_span_location(corpus_l, span, kw_span, verbose=False)
    first_locations[kw] = location

print(story)
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(first_locations)

### The Boscombe Valley Mystery

In [ ]:
story = 'The Boscombe Valley Mystery'

# Get corpus span
corpus = get_corpus(story)
corpus_l = corpus.lower()
spans = get_adventures(corpus_l, n=4)
span = spans[0]

# Full story
story_spans, story_counts = crime_story(CRIME_KEYWORDS, corpus_l, span)

pp = pprint.PrettyPrinter(indent=2)
#pp.pprint(story_spans)
pp.pprint(story_counts)


# Get span of first occurrence
first_occurrences = get_first_crime(story_spans)
# print(first_occurrences)

# Find location of first span
first_locations = {}
for kw, kw_span in first_occurrences.items():
    location = get_span_location(corpus_l, span, kw_span, verbose=False)
    first_locations[kw] = location

print(story)
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(first_locations)

#### Utility functions for location of crime

In [ ]:
# Identify paragraph and sentence based on a keyword.
# This is used for crime detection analysis.
search_pair = ['blood', 'mysterious']
gp = 0
found = False

c = get_chapters(text, span)
for k, _c in enumerate(c, start=1):
    p = get_paragraphs(text, _c)
    for i, _p in enumerate(p, start=1):
        gp += 1
        s = get_sentences(text, _p)
        for j, _s in enumerate(s, start=1):
            print(f'C{k}, GP{gp}, S{j}')
            t = get_text_from_span(text, _s)
            print(f'C{k}, P{i}, S{j} ------------------------\n', t)
            if search_pair[0] in t and search_pair[1] in t:
                found = True
                break
        if found: break
    if found: break

In [ ]:
# Identify paragraph and sentence based on a keyword.
# This is used for crime detection analysis.
search_pair = ['murdered', 'man']
found = False

p = get_paragraphs(text, span)
for i, _p in enumerate(p, start=1):
    s = get_sentences(text, _p)
    for j, _s in enumerate(s, start=1):
        t = get_text_from_span(text, _s)
        if search_pair[0] in t and search_pair[1] in t:
            print(f'P{i}, S{j} ------------------------\n', t)
            found = True
            break
    if found: break

## Organize data for Jerry's visualization tool

In [ ]:
# NOTE: Run section 'Crime Detection' first

data = []

item = collections.OrderedDict()
item['title'] = story
item['author'] = 'Sir Arthur Conan Doyle'
item['queryType'] = 'occurrences'
item['question'] = 'Crime'
item['numChapters'] = len(story_counts)

results = collections.defaultdict(collections.OrderedDict)
for chapter, word_map in story_counts.items():
    for word, freq in word_map.items():
        results[word][str(chapter)] = freq
        
for word, freq_map in results.items():
    if sum(freq_map.values()) < 5:
        continue
    item['query'] = word
    item['results'] = freq_map
    data.append(copy.deepcopy(item))

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(data)

# Story structure

In [ ]:
def count_text_structure(text, span=None, structure=None):
    if structure is None:
        structure = collections.defaultdict(int)

    for par_span in get_paragraphs(text, span):
        structure['Paragraphs'] += 1
        for sent_span in get_sentences(text, par_span):
            structure['Sentences'] += 1
            for tok_span in get_tokens(text, sent_span):
                structure['Words'] += 1
    return structure

In [ ]:
story = 'The Valley of Fear'
corpus = get_corpus(story)
text = corpus.lower()
structure = collections.defaultdict(int)
for part_span in get_parts(text):
    structure['Parts'] += 1
    for chp_span in get_chapters(text, part_span):
        structure['Chapters'] += 1
        count_text_structure(text, chp_span, structure)
structure['Epilogue'] = 1
span = get_text(text)[0]
structure['Characters'] = span[1] - span[0]
count_text_structure(text, get_epilogue(text)[0], structure)

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(structure)

In [ ]:
story = 'A Study in Scarlet'
corpus = get_corpus(story)
text = corpus.lower()
structure = collections.defaultdict(int)
for part_span in get_parts(text):
    structure['Parts'] += 1
    for chp_span in get_chapters(text, part_span):
        structure['Chapters'] += 1
        count_text_structure(text, chp_span, structure)
span = get_text(text)[0]
structure['Characters'] = span[1] - span[0]

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(structure)

In [ ]:
story = 'The Sign of the Four'
corpus = get_corpus(story)
text = corpus.lower()
structure = collections.defaultdict(int)
for chp_span in get_chapters(text):
    structure['Chapters'] += 1
    count_text_structure(text, chp_span, structure)
span = get_text(text)[0]
structure['Characters'] = span[1] - span[0]

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(structure)

In [ ]:
story = 'The Hound of the Baskervilles'
corpus = get_corpus(story)
text = corpus.lower()
structure = collections.defaultdict(int)
for chp_span in get_chapters(text):
    structure['Chapters'] += 1
    count_text_structure(text, chp_span, structure)
span = get_text(text)[0]
structure['Characters'] = span[1] - span[0]

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(structure)

In [ ]:
story = 'The Boscombe Valley Mystery'
corpus = get_corpus(story)
text = corpus.lower()
structure = collections.defaultdict(int)
span = get_adventures(text, n=4)[0]
count_text_structure(text, span, structure)
structure['Characters'] = span[1] - span[0]

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(structure)

# Find character entities

In [ ]:
story = 'The Valley of Fear'

# Select part(s)
# spans = get_parts(get_corpus(story).lower(), n=1)  # n = Part
# span = spans[0]  # [i] = Part i+1
span = None  # all Parts or if no Parts

# Per chapter
story_spans, story_counts = ner_story_with_chapters(story, span)
plot_ner_counts_story_with_chapters(story, story_counts, show=False)

# Full story
# story_spans, story_counts = ner_story(story, span)
# plot_ner_counts_story(story, story_counts, show=False)

pp = pprint.PrettyPrinter(indent=2)
# pp.pprint(story_spans)
pp.pprint(story_counts)

In [ ]:
story = 'A Study in Scarlet'

# Select part(s)
spans = get_parts(get_corpus(story).lower())  # n = Part
span = spans[0]  # [i] = Part i+1

# Per chapter
story_spans, story_counts = ner_story_with_chapters(story, span)
plot_ner_counts_story_with_chapters(story, story_counts, show=True)

# Full story
# story_spans, story_counts = ner_story(story, span)
# plot_ner_counts_story(story, story_counts, show=True)

pp = pprint.PrettyPrinter(indent=2)
# pp.pprint(story_spans)
pp.pprint(story_counts)

In [ ]:
story = 'The Sign of the Four'

# Per chapter
story_spans, story_counts = ner_story_with_chapters(story)
plot_ner_counts_story_with_chapters(story, story_counts, show=True)

# Full story
# story_spans, story_counts = ner_story(story)
# plot_ner_counts_story(story, story_counts, show=True)

pp = pprint.PrettyPrinter(indent=2)
# pp.pprint(story_spans)
pp.pprint(story_counts)

In [ ]:
story = 'The Hound of the Baskervilles'

# Per chapter
story_spans, story_counts = ner_story_with_chapters(story)
plot_ner_counts_story_with_chapters(story, story_counts, show=True)

# Full story
# story_spans, story_counts = ner_story(story)
# plot_ner_counts_story(story, story_counts, show=True)

pp = pprint.PrettyPrinter(indent=2)
# pp.pprint(story_spans)
pp.pprint(story_counts)

In [ ]:
story = 'The Boscombe Valley Mystery'
spans = get_adventures(get_corpus(story).lower(), n=4)
span = spans[0]

# Per paragraph
# story_spans, story_counts = ner_story_with_paragraphs(story, span)
# plot_ner_counts_story_with_paragraphs(story, story_counts, show=True)

# Full story
story_spans, story_counts = ner_story(story, span)
plot_ner_counts_story(story, story_counts, show=True)

pp = pprint.PrettyPrinter(indent=2)
# pp.pprint(story_spans)
pp.pprint(story_counts)

## Organize data for Jerry's visualization tool

In [ ]:
# NOTE: Run section 'Find character entities' first

data = []
title = story
author = 'Sir Arthur Conan Doyle'
query_type = 'occurrences'
for character_type, chapter_map in story_counts.items():
    if character_type == 'detectives':
        question = 'Detective'
    elif character_type == 'perpetrators':
        question = 'Perpetrator'
    elif character_type == 'suspects':
        question = 'Other Suspects'
    else:
        continue
    
    item = collections.OrderedDict()
    item['title'] = title
    item['author'] = author
    item['queryType'] = query_type
    item['question'] = question
    item['numChapters'] = len(chapter_map)

    characters = CHARACTERS_NAMES[story][character_type]

    results = collections.defaultdict(collections.OrderedDict)
    aliases = {}
    for chapter, character_map in chapter_map.items():
        for character, freq in character_map.items():
            if character not in results:
                for c in characters:
                    if c[0] == character:
                        aliases[character] = '|'.join([s.replace('_', ' ') for s in c])
            results[character][str(chapter)] = freq
            
    for character, freq_map in results.items():
        item['query'] = aliases[character]
        item['results'] = freq_map
        data.append(copy.deepcopy(item))

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(data)

## Identify first-time occurrences

### The Valley of Fear, The Sign of the Four, The Hound of the Baskervilles

In [ ]:
print(story)

# Get span of first occurrence
first_occurrences = get_first_occurrences(story_spans)
# print(first_occurrences)

# Get corpus span
corpus_l = get_corpus(story).lower()
spans = get_text(corpus_l)
span = spans[0]

# Find location of first span
first_locations = {}
for character_type, characters in first_occurrences.items():
    first_locations[character_type] = {}
    for character in characters:
        location = get_span_location_with_chapters(corpus_l, span, first_occurrences[character_type][character], verbose=False)
        first_locations[character_type][character] = location

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(first_locations)

### A Study in Scarlet

In [ ]:
print(story)

# Get span of first occurrence
first_occurrences = get_first_occurrences(story_spans)
# print(first_occurrences)

# Get corpus span
text = get_corpus(story).lower()
spans = get_parts(text)  # n = Part
span = spans[0]  # [i] = Part i+1

# Find location of first span
first_locations = {}
for character_type, characters in first_occurrences.items():
    first_locations[character_type] = {}
    for character in characters:
        location = get_span_location_with_chapters(text, span, first_occurrences[character_type][character], verbose=False)
        first_locations[character_type][character] = location

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(first_locations)

### The Boscombe Valley Mystery

In [ ]:
print(story)

# Get span of first occurrence
first_occurrences = get_first_occurrences(story_spans)
# print(first_occurrences)

# Get corpus span
text = get_corpus(story).lower()
spans = get_adventures(text, n=4)
span = spans[0]

# Find location of first span
first_locations = {}
for character_type, characters in first_occurrences.items():
    first_locations[character_type] = {}
    for character in characters:
        location = get_span_location(text, span, first_occurrences[character_type][character], verbose=False)
        first_locations[character_type][character] = location

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(first_locations)

## Neighbor words

In [ ]:
# NOTE: Need to run "Find character entities" as an initial step.

print(story_spans)

# Merge perpetrators spans across story chapters/sections
perpetrators = {}
for section, characters in story_spans['perpetrators'].items():
    for character, spans in characters.items():
        if character not in perpetrators:
            perpetrators[character] = []
        perpetrators[character].extend(spans)

In [ ]:
corpus = get_corpus(story)
corpus_l = corpus.lower()
spans = get_text(corpus_l)
#spans = get_chapters(corpus_l)
#spans = get_adventures(get_corpus(story).lower(), n=4)

tok_spans = get_tokens(corpus_l, spans[0])

print(len(tok_spans))

In [ ]:
# Method 1: No filtering
perpetrator_neighbors = get_neighbor_words_for_character(corpus_l, tok_spans, perpetrators, CHARACTERS_NAMES[story]['perpetrators'])
pp.pprint(perpetrator_neighbors)

In [ ]:
# Method 2: Filter stopwords
perpetrator_neighbors_stop = get_neighbor_words_for_character(corpus_l, tok_spans, perpetrators, CHARACTERS_NAMES[story]['perpetrators'], stopwords=STOPWORDS)
pp.pprint(perpetrator_neighbors_stop)

In [ ]:
# Plot top neighboring words
top_n = 10

for character, freq_modes in perpetrator_neighbors_stop.items():
    for mode, freq_all in freq_modes.items():
        freq = get_frequent_items(freq_all, top_n)
        print(freq)
        if freq:
            barplot(list(freq.values()), list(freq.keys()))
            plt.show()

## Organize data for Jerry's visualization tool

In [ ]:
data = []

item = collections.OrderedDict()
item['title'] = story
item['author'] = 'Sir Arthur Conan Doyle'
item['queryType'] = 'nearby'
item['question'] = 'Nearby Words'
item['numChapters'] = len(story_counts['perpetrators'])

characters = CHARACTERS_NAMES[story]['perpetrators']

aliases = {}
for character, direction_map in perpetrator_neighbors_stop.items():
    for c in characters:
        if c[0] == character:
            item['query'] = '|'.join([s.replace('_', ' ') for s in c])
            break
    for direction, freq_map in direction_map.items():
        counter = collections.Counter(freq_map)
        item[direction] = counter.most_common(3)
    data.append(copy.deepcopy(item))

pp = pprint.PrettyPrinter(indent=2)
pp.pprint(data)

# EOF